In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
user_event = pd.read_csv('/Users/t2023-m0017/Desktop/실습/프로젝트/최종프로젝트/user_event.csv',parse_dates=['created_at_x','created_at_y'])
events = pd.read_csv( '/Users/t2023-m0017/Desktop/실습/프로젝트/최종프로젝트/3차/events.csv',parse_dates=['created_at'])
users = pd.read_csv('/Users/t2023-m0017/Desktop/실습/프로젝트/최종프로젝트/3차/users.csv')
order_items = pd.read_csv( '/Users/t2023-m0017/Desktop/실습/프로젝트/최종프로젝트/3차/order_items.csv')
products = pd.read_csv('/Users/t2023-m0017/Desktop/실습/프로젝트/최종프로젝트/3차/products.csv')

/var/folders/1y/dmb1rjqn7y7ft_t08s501gxm0000gn/T/ipykernel_10158/96766544.py:1: DtypeWarning: Columns (2,3,4,6,7,8,9,10,11,14,23) have mixed types. Specify dtype option on import or set low_memory=False.
  user_event = pd.read_csv('/Users/t2023-m0017/Desktop/실습/프로젝트/최종프로젝트/user_event.csv',parse_dates=['created_at_x','created_at_y'])


In [4]:
buy_user = events[events['event_type']=='purchase']

In [5]:
buy_user.head(1)

,id,user_id,sequence_number,session_id,created_at,ip_address,city,state,postal_code,browser,traffic_source,uri,event_type
23067,808414,61899.0,13,1daacb69-a4b6-4ef6-8686-bc62f86e87d5,2023-02-25 01:33:38+00:00,191.109.103.15,São Paulo,São Paulo,02220-000,Chrome,Facebook,/purchase,purchase


In [6]:
import datetime 
buy_user['created_at'] = buy_user['created_at'].dt.normalize()
buy_user['year'] = buy_user['created_at'].dt.year
buy_user['month'] = buy_user['created_at'].dt.month


/var/folders/1y/dmb1rjqn7y7ft_t08s501gxm0000gn/T/ipykernel_10158/2812700201.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_user['created_at'] = buy_user['created_at'].dt.normalize()
/var/folders/1y/dmb1rjqn7y7ft_t08s501gxm0000gn/T/ipykernel_10158/2812700201.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_user['year'] = buy_user['created_at'].dt.year
/var/folders/1y/dmb1rjqn7y7ft_t08s501gxm0000gn/T/ipykernel_10158/2812700201.py:4: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [7]:
total_user = events.copy()

In [8]:
total_user['created_at'] = total_user['created_at'].dt.normalize()
total_user['year'] = total_user['created_at'].dt.year
total_user['month'] = total_user['created_at'].dt.month

In [9]:
# 재주문한 사람도 있으니까 userid를 카운트
buy_traffic_cnt = buy_user.groupby(['year','month','traffic_source']).agg({'user_id':'count'}).reset_index()
# 월별 총 마케팅 비용을 구하기 위해 트래픽이 있는 모든 session_id를 nunique
total_traffic_cnt = total_user.groupby(['year','month','traffic_source']).agg({'session_id':'nunique'}).reset_index()

In [56]:
buy_traffic_cnt

,year,month,traffic_source,user_id
0,2019,1,Adwords,11
1,2019,1,Email,12
2,2019,1,Facebook,2
3,2019,1,Organic,1
4,2019,1,YouTube,6
...,...,...,...,...
300,2024,1,Adwords,4072
301,2024,1,Email,6196
302,2024,1,Facebook,1382
303,2024,1,Organic,687


In [10]:
# adwords
cost_adwords = total_traffic_cnt[total_traffic_cnt['traffic_source'] == "Adwords"].reset_index(drop=True)
cost_adwords.rename(columns={'session_id':'visits'},inplace = True)
cost_adwords['cost'] = cost_adwords['visits']*2.25
# 월 구매건수 구하기
buy_adwords = buy_traffic_cnt[buy_traffic_cnt['traffic_source']=='Adwords'].reset_index(drop=True)
buy_adwords.rename(columns={'user_id':'buyer'},inplace=True)
#cac
cost_adwords['cac'] = cost_adwords ['cost']/buy_adwords['buyer']

In [57]:
cost_adwords['cac']

KeyError: 'cac'

In [12]:
# Facebook
cost_Facebook = total_traffic_cnt[total_traffic_cnt['traffic_source'] == "Facebook"].reset_index(drop=True)
cost_Facebook.rename(columns={'session_id':'visits'},inplace = True)
cost_Facebook['cost'] = cost_Facebook['visits']*0.69
# 월 구매건수 구하기
buy_Facebook = buy_traffic_cnt[buy_traffic_cnt['traffic_source']=='Facebook'].reset_index(drop=True)
buy_Facebook.rename(columns={'user_id':'buyer'},inplace=True)
#cac
cost_Facebook['cac'] = cost_Facebook['cost']/buy_Facebook['buyer']

In [13]:
cost_Facebook

,year,month,traffic_source,visits,cost,cac
0,2019,1,Facebook,821,566.49,283.245000
1,2019,2,Facebook,769,530.61,75.801429
2,2019,3,Facebook,823,567.87,56.787000
3,2019,4,Facebook,845,583.05,41.646429
4,2019,5,Facebook,854,589.26,34.662353
...,...,...,...,...,...,...
56,2023,9,Facebook,1621,1118.49,1.424828
57,2023,10,Facebook,1797,1239.93,1.323298
58,2023,11,Facebook,1883,1299.27,1.317718
59,2023,12,Facebook,2115,1459.35,1.223261


In [14]:
#  'YouTube'
cost_YouTube = total_traffic_cnt[total_traffic_cnt['traffic_source'] == "YouTube"].reset_index(drop=True)
cost_YouTube.rename(columns={'session_id':'visits'},inplace = True)
cost_YouTube['cost'] = cost_YouTube['visits']*0.03
# 월 구매건수 구하기
buy_YouTube = buy_traffic_cnt[buy_traffic_cnt['traffic_source']=='YouTube'].reset_index(drop=True)
buy_YouTube.rename(columns={'user_id':'buyer'},inplace=True)
#cac
cost_YouTube['cac'] = cost_YouTube['cost']/buy_YouTube['buyer']

In [15]:
cost_YouTube

,year,month,traffic_source,visits,cost,cac
0,2019,1,YouTube,787,23.61,3.935000
1,2019,2,YouTube,755,22.65,5.662500
2,2019,3,YouTube,815,24.45,1.880769
3,2019,4,YouTube,800,24.00,1.600000
4,2019,5,YouTube,851,25.53,1.276500
...,...,...,...,...,...,...
56,2023,9,YouTube,1607,48.21,0.062448
57,2023,10,YouTube,1811,54.33,0.058232
58,2023,11,YouTube,1774,53.22,0.055669
59,2023,12,YouTube,2149,64.47,0.051783


In [16]:
#  'Email'
cost_Email = total_traffic_cnt[total_traffic_cnt['traffic_source'] == "Email"].reset_index(drop=True)
cost_Email.rename(columns={'session_id':'visits'},inplace = True)
cost_Email['cost'] = 1000
# 월 구매건수 구하기
buy_Email = buy_traffic_cnt[buy_traffic_cnt['traffic_source']=='Email'].reset_index(drop=True)
buy_Email.rename(columns={'user_id':'buyer'},inplace=True)
#cac
cost_Email['cac'] = cost_Email['cost']/buy_Email['buyer']

In [17]:
cost_Email

,year,month,traffic_source,visits,cost,cac
0,2019,1,Email,3529,1000,83.333333
1,2019,2,Email,3491,1000,47.619048
2,2019,3,Email,3840,1000,13.888889
3,2019,4,Email,3665,1000,14.492754
4,2019,5,Email,3954,1000,8.264463
...,...,...,...,...,...,...
56,2023,9,Email,7332,1000,0.285470
57,2023,10,Email,8015,1000,0.251067
58,2023,11,Email,8284,1000,0.224871
59,2023,12,Email,9486,1000,0.186672


In [18]:
cost_adwords
# cost_Email
# cost_Facebook
# cost_YouTube

,year,month,traffic_source,visits,cost,cac
0,2019,1,Adwords,2423,5451.75,495.613636
1,2019,2,Adwords,2328,5238.00,261.900000
2,2019,3,Adwords,2601,5852.25,292.612500
3,2019,4,Adwords,2588,5823.00,123.893617
4,2019,5,Adwords,2622,5899.50,85.500000
...,...,...,...,...,...,...
56,2023,9,Adwords,4908,11043.00,4.622436
57,2023,10,Adwords,5212,11727.00,4.536557
58,2023,11,Adwords,5517,12413.25,4.186594
59,2023,12,Adwords,6227,14010.75,3.869304


In [19]:
cost_Email.rename(columns={'cac':'email_cac'},inplace=True)
cost_adwords.rename(columns={'cac':'adwords_cac'},inplace=True)
cost_Facebook.rename(columns={'cac':'Facebook_cac'},inplace=True)
cost_YouTube.rename(columns={'cac':'YouTube_cac'},inplace=True)

In [20]:
cost_Email

,year,month,traffic_source,visits,cost,email_cac
0,2019,1,Email,3529,1000,83.333333
1,2019,2,Email,3491,1000,47.619048
2,2019,3,Email,3840,1000,13.888889
3,2019,4,Email,3665,1000,14.492754
4,2019,5,Email,3954,1000,8.264463
...,...,...,...,...,...,...
56,2023,9,Email,7332,1000,0.285470
57,2023,10,Email,8015,1000,0.251067
58,2023,11,Email,8284,1000,0.224871
59,2023,12,Email,9486,1000,0.186672


In [21]:
df_cac = pd.concat([cost_Email["year"],cost_Email['month'],cost_Email['email_cac'],cost_adwords['adwords_cac'],cost_Facebook['Facebook_cac'],cost_YouTube['YouTube_cac']],axis = 1,ignore_index=False)

In [22]:
cost_adwords['adwords_cac'],cost_Facebook['Facebook_cac'],cost_YouTube['YouTube_cac']

(0     495.613636
 1     261.900000
 2     292.612500
 3     123.893617
 4      85.500000
          ...    
 56      4.622436
 57      4.536557
 58      4.186594
 59      3.869304
 60      3.020813
 Name: adwords_cac, Length: 61, dtype: float64,
 0     283.245000
 1      75.801429
 2      56.787000
 3      41.646429
 4      34.662353
          ...    
 56      1.424828
 57      1.323298
 58      1.317718
 59      1.223261
 60      0.889711
 Name: Facebook_cac, Length: 61, dtype: float64,
 0     3.935000
 1     5.662500
 2     1.880769
 3     1.600000
 4     1.276500
         ...   
 56    0.062448
 57    0.058232
 58    0.055669
 59    0.051783
 60    0.039558
 Name: YouTube_cac, Length: 61, dtype: float64)

In [23]:
df_cac 

,year,month,email_cac,adwords_cac,Facebook_cac,YouTube_cac
0,2019,1,83.333333,495.613636,283.245000,3.935000
1,2019,2,47.619048,261.900000,75.801429,5.662500
2,2019,3,13.888889,292.612500,56.787000,1.880769
3,2019,4,14.492754,123.893617,41.646429,1.600000
4,2019,5,8.264463,85.500000,34.662353,1.276500
...,...,...,...,...,...,...
56,2023,9,0.285470,4.622436,1.424828,0.062448
57,2023,10,0.251067,4.536557,1.323298,0.058232
58,2023,11,0.224871,4.186594,1.317718,0.055669
59,2023,12,0.186672,3.869304,1.223261,0.051783


In [24]:
fig = px.line(df_cac , x = 'month' ,y = 'email_cac', width = 1200, height = 500, color='year', title =" Email CAC")
fig.show()

In [25]:
fig = px.line(df_cac , x = 'month' ,y = 'adwords_cac', width = 1200, height = 500, color='year',title = 'adwords_cac')
fig.show()

In [26]:
fig = px.line(df_cac , x = 'month' ,y = 'YouTube_cac', width = 1200, height = 500, color='year', title ='YouTube_cac')
fig.show()

In [27]:
fig = px.line(df_cac , x = 'month' ,y = 'Facebook_cac', width = 1200, height = 500, color='year', title = 'Facebook_cac')
fig.show()

### 구매전환율

In [28]:
Facebook_con = pd.concat([cost_Facebook,buy_Facebook[['buyer']]],axis = 1,ignore_index=False)
Email_con= pd.concat([cost_Email,buy_Email[['buyer']]],axis = 1,ignore_index=False)
YouTube_con = pd.concat([cost_YouTube,buy_YouTube[['buyer']]],axis = 1,ignore_index=False)
Adwords_con = pd.concat([cost_adwords,buy_adwords[['buyer']]],axis = 1,ignore_index=False)

In [29]:
Facebook_con['rate'] =Facebook_con['buyer']/Facebook_con['visits']*100
Email_con['rate'] =Email_con['buyer']/Email_con['visits']*100
YouTube_con['rate'] =YouTube_con['buyer']/YouTube_con['visits']*100
Adwords_con['rate'] =Adwords_con['buyer']/Adwords_con['visits']*100

In [30]:
#  'Organic'
cost_Organic = total_traffic_cnt[total_traffic_cnt['traffic_source'] == "Organic"].reset_index(drop=True)
cost_Organic.rename(columns={'session_id':'visits'},inplace = True)

# 월 구매건수 구하기
buy_Organic = buy_traffic_cnt[buy_traffic_cnt['traffic_source']=='Organic'].reset_index(drop=True)
buy_Organic.rename(columns={'user_id':'buyer'},inplace=True)

Organic_con = pd.concat([cost_Organic,buy_Organic[['buyer']]],axis = 1,ignore_index=False)
Organic_con['rate'] =Organic_con['buyer']/Organic_con['visits']*100
Organic_con

,year,month,traffic_source,visits,buyer,rate
0,2019,1,Organic,413,1,0.242131
1,2019,2,Organic,390,1,0.256410
2,2019,3,Organic,433,8,1.847575
3,2019,4,Organic,411,7,1.703163
4,2019,5,Organic,450,22,4.888889
...,...,...,...,...,...,...
56,2023,9,Organic,858,420,48.951049
57,2023,10,Organic,878,448,51.025057
58,2023,11,Organic,960,505,52.604167
59,2023,12,Organic,1039,589,56.689124


In [31]:
Email_con

,year,month,traffic_source,visits,cost,email_cac,buyer,rate
0,2019,1,Email,3529,1000,83.333333,12,0.340040
1,2019,2,Email,3491,1000,47.619048,21,0.601547
2,2019,3,Email,3840,1000,13.888889,72,1.875000
3,2019,4,Email,3665,1000,14.492754,69,1.882674
4,2019,5,Email,3954,1000,8.264463,121,3.060192
...,...,...,...,...,...,...,...,...
56,2023,9,Email,7332,1000,0.285470,3503,47.776869
57,2023,10,Email,8015,1000,0.251067,3983,49.694323
58,2023,11,Email,8284,1000,0.224871,4447,53.681796
59,2023,12,Email,9486,1000,0.186672,5357,56.472697


In [32]:
buy_user['traffic_source'].unique()

array(['Facebook', 'Email', 'Adwords', 'YouTube', 'Organic'], dtype=object)

In [33]:
fig = px.line(Organic_con , x = 'month' ,y = 'rate', color='year', title = 'Organic_con')
fig.show()

In [34]:
fig = px.line(Facebook_con , x = 'month' ,y = 'rate', color='year', title = 'Facebook_con')
fig.show()

In [35]:
fig = px.line(Email_con , x = 'month' ,y = 'rate', color='year', title = 'Email_con')
fig.show()

In [36]:
fig = px.line(Adwords_con , x = 'month' ,y = 'rate', color='year', title = 'Adwords_con')
fig.show()

### 가입시점에 따른 결제 전환율

In [37]:
# 날짜별 결제자수
user_event = user_event.drop('Unnamed: 0',axis = 1)
user_event_buy = user_event[user_event['event_type']=='purchase']
user_event_buy['created_at_x'] = user_event_buy['created_at_x'].dt.normalize()
user_event_buy ['created_at_y'] = user_event_buy['created_at_y'].dt.normalize()


/var/folders/1y/dmb1rjqn7y7ft_t08s501gxm0000gn/T/ipykernel_10158/161830139.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/1y/dmb1rjqn7y7ft_t08s501gxm0000gn/T/ipykernel_10158/161830139.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [38]:
user_event_buy.rename(columns={'created_at_x':'join_day','created_at_y':'buy_day'},inplace=True)
user_event_buy_df = user_event_buy.groupby(['join_day']).agg({'user_id':'count'})
user_event_buy_df.rename(columns={'user_id':'buyer'},inplace=True)

/var/folders/1y/dmb1rjqn7y7ft_t08s501gxm0000gn/T/ipykernel_10158/2504011481.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [39]:
# 날짜별 가입자수
user_event_total = user_event
user_event_total['created_at_x'] = user_event_total['created_at_x'].dt.normalize()
user_event_total['created_at_y'] =user_event_total['created_at_y'].dt.normalize()
user_event_total.rename(columns={'created_at_x':'join_day','created_at_y':'buy_day'},inplace=True)
user_event_total_df = user_event_total.groupby(['join_day']).agg({'user_id':'count'})
user_event_total_df.rename(columns={'user_id':'join'},inplace=True)

In [40]:
user_event_total_df

,join
join_day,
2019-01-02 00:00:00+00:00,436
2019-01-03 00:00:00+00:00,810
2019-01-04 00:00:00+00:00,587
2019-01-05 00:00:00+00:00,819
2019-01-06 00:00:00+00:00,629
...,...
2024-01-12 00:00:00+00:00,6568
2024-01-13 00:00:00+00:00,6437
2024-01-14 00:00:00+00:00,6303


In [41]:
# 가입날짜별 구매건수
user_event_buy_df


,buyer
join_day,
2019-01-02 00:00:00+00:00,66
2019-01-03 00:00:00+00:00,106
2019-01-04 00:00:00+00:00,85
2019-01-05 00:00:00+00:00,117
2019-01-06 00:00:00+00:00,88
...,...
2024-01-12 00:00:00+00:00,893
2024-01-13 00:00:00+00:00,863
2024-01-14 00:00:00+00:00,870


In [46]:
df_day = pd.concat([user_event_total_df,user_event_buy_df],axis =1)
df_day['rate']=df_day['join']/df_day['buyer']*100

In [48]:
df_day.reset_index(inplace=True)

In [54]:
df_day.pivot('')

,join_day,join,buyer,rate
0,2019-01-02 00:00:00+00:00,436,66,6.606061
1,2019-01-03 00:00:00+00:00,810,106,7.641509
2,2019-01-04 00:00:00+00:00,587,85,6.905882
3,2019-01-05 00:00:00+00:00,819,117,7.000000
4,2019-01-06 00:00:00+00:00,629,88,7.147727
...,...,...,...,...
1836,2024-01-12 00:00:00+00:00,6568,893,7.354983
1837,2024-01-13 00:00:00+00:00,6437,863,7.458864
1838,2024-01-14 00:00:00+00:00,6303,870,7.244828
1839,2024-01-15 00:00:00+00:00,5673,804,7.055970


In [53]:
df_day.describe()

,join,buyer,rate
count,1841.000000,1841.000000,1841.000000
mean,720.395437,98.728408,7.270878
std,344.886117,46.164994,0.493569
min,286.000000,37.000000,5.745098
25%,594.000000,84.000000,6.928571
50%,697.000000,96.000000,7.268041
75%,805.000000,108.000000,7.615385
max,6568.000000,893.000000,9.177215


In [52]:
fig = px.line(df_day, x = 'join_day' ,y = 'rate',  title = '가입날짜에 따른 구매율')
fig.show()